In [1]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [2]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
#创建一个给定形状的张量，但元素内容是未初始化的（内存中的随机值、垃圾值）。
#如果你要“马上填充所有元素”，如循环赋值或从数据加载，则torch.empty效率更高。
for i, x_i in  enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i,query)
#A 第二个输入 token 用作查询向量
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
attn_weights_2_tmp = attn_scores_2 / (attn_scores_2.sum())
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())
# 归一化的主要目的是使注意力权重之和为 1。
# 这种归一化是一种有助于解释和保持LLM训练稳定性的惯例

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [4]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())
# 常见且更推荐使用 softmax 函数来进行归一化。
# 这种方法更擅长处理极端值，并且在训练过程中提供了更有利的梯度特性
# 此外，softmax 函数确保注意力权重始终为正值。
# 这使得输出可以被解释为概率或相对重要性，其中较高的权重表示更重要。

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [5]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [6]:
query = inputs[1]
context_vector_2 = torch.zeros(query.shape)
for i ,xi in enumerate(attn_weights_2):
    context_vector_2 += xi *inputs[i]
print(context_vector_2)
# 上下文向量 z(2) 是所有输入向量的加权和。
# 其计算方法为将每个输入向量与对应的注意力权重相乘后相加。


tensor([0.4419, 0.6515, 0.5683])


In [7]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [8]:
attn_scores = torch.empty(6, 6)
# 6个输入 相互计算 就是6*6
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [9]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
attn_weights = torch.softmax(attn_scores,dim = -1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [11]:
sum(attn_weights[1])

tensor(1.)

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

其实就是每个位置的向量 单独与其他所有位置的向量进行点积，计算出来的值再进行归一化就得到了位置i与其它所有位置的权重，然后再通过将该权重和每个位置的向量进行乘积再相加就得到了上下文向量。

## 3.4 实现带有可训练权重的自注意力机制

In [14]:
x_2 = inputs[1]                                                   #A
d_in = inputs.shape[1]                                            #B
d_out = 2                                                         #C

# 请注意，在 GPT 类模型中，输入维度和输出维度通常是相同的。
# 不过，为了便于说明和更清楚地展示计算过程，
# 我们在此选择了不同的输入（d_in=3）和输出（d_out=2）维度。
#A 第二个输入元素
#B 输入维度， d_in=3
#C 输出维度， d_out=2

In [15]:
torch.manual_seed(42)

In [16]:
w_q = torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad = True)
w_k = torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad = True)
w_v = torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad = True)
w_q,w_k,w_v

(Parameter containing:
 tensor([[ 0.3367,  0.1288],
         [ 0.2345,  0.2303],
         [-1.1229, -0.1863]], requires_grad=True),
 Parameter containing:
 tensor([[ 2.2082, -0.6380],
         [ 0.4617,  0.2674],
         [ 0.5349,  0.8094]], requires_grad=True),
 Parameter containing:
 tensor([[ 1.1103, -1.6898],
         [-0.9890,  0.9580],
         [ 1.3221,  0.8172]], requires_grad=True))

In [17]:
q2 = inputs[1] @ w_q
k2 = inputs[1] @ w_k
v2 = inputs[1] @ w_v
q2,k2,v2

(tensor([-0.3519,  0.1483], grad_fn=<SqueezeBackward4>),
 tensor([1.9692, 0.4159], grad_fn=<SqueezeBackward4>),
 tensor([0.6229, 0.4434], grad_fn=<SqueezeBackward4>))

In [18]:
inputs,inputs.shape

(tensor([[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]),
 torch.Size([6, 3]))

In [19]:
key = inputs @ w_k
value = inputs @ w_v
key,value,key.shape,value.shape

(tensor([[ 1.4948,  0.4861],
         [ 1.9692,  0.4159],
         [ 1.9934,  0.3816],
         [ 0.9301,  0.2818],
         [ 1.8692, -0.3435],
         [ 0.7739,  0.6271]], grad_fn=<MmBackward0>),
 tensor([[ 1.5058,  0.1444],
         [ 0.6229,  0.4434],
         [ 0.6384,  0.3741],
         [ 0.1070,  0.4535],
         [ 0.7399, -0.9799],
         [-0.0085,  1.1313]], grad_fn=<MmBackward0>),
 torch.Size([6, 2]),
 torch.Size([6, 2]))

In [20]:
attn_scores_2 = q2 @ key.T
attn_scores_2


tensor([-0.4540, -0.6313, -0.6450, -0.2855, -0.7087, -0.1794],
       grad_fn=<SqueezeBackward4>)

In [21]:
attn_scores_2 = torch.softmax(attn_scores_2 / (key.shape[-1]**0.5),dim=-1)
attn_scores_2

tensor([0.1686, 0.1487, 0.1473, 0.1899, 0.1408, 0.2047],
       grad_fn=<SoftmaxBackward0>)

In [22]:
context_vec_2 = attn_scores_2 @ value
context_vec_2

tensor([0.5633, 0.3251], grad_fn=<SqueezeBackward4>)

In [23]:
import torch
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.wk = nn.Parameter(torch.randn(d_in, d_out))
        self.wq = nn.Parameter(torch.randn(d_in, d_out))
        self.wv = nn.Parameter(torch.randn(d_in, d_out))
        self.d_out = d_out

    def forward(self, x):
        # x: [seq_len, d_in]
        query = x @ self.wq    # [seq_len, d_out]
        key = x @ self.wk      # [seq_len, d_out]
        value = x @ self.wv    # [seq_len, d_out]
        attn_scores = query @ key.T   # [seq_len, seq_len]
        attn_weights = torch.softmax(attn_scores / (self.d_out ** 0.5), dim=-1)
        context = attn_weights @ value   # [seq_len, d_out]
        return context


In [24]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in,d_out)
print(sa_v1(inputs))

tensor([[0.2904, 0.3920],
        [0.2903, 0.3950],
        [0.2905, 0.3942],
        [0.2880, 0.3932],
        [0.2927, 0.3798],
        [0.2864, 0.4014]], grad_fn=<MmBackward0>)


In [25]:
#  Listing 3.2 A self-attention class using PyTorch's Linear layers
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [26]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [27]:
print(sa_v1.wk)
print(sa_v1.wq)
print(sa_v1.wv)


Parameter containing:
tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]], requires_grad=True)
Parameter containing:
tensor([[-0.9724, -0.7550],
        [ 0.3239, -0.1085],
        [ 0.2103, -0.3908]], requires_grad=True)
Parameter containing:
tensor([[ 0.2350,  0.6653],
        [ 0.3528,  0.9728],
        [-0.0386, -0.8861]], requires_grad=True)


In [28]:
print(sa_v2.W_query.weight)
print(sa_v2.W_key.weight)
print(sa_v2.W_value.weight)
# 使用nn.linear输出的权重是自带转置的

Parameter containing:
tensor([[ 0.3161,  0.4568,  0.5118],
        [-0.1683, -0.3379, -0.0918]], requires_grad=True)
Parameter containing:
tensor([[ 0.4058, -0.4704,  0.2368],
        [ 0.2134, -0.2601, -0.5105]], requires_grad=True)
Parameter containing:
tensor([[ 0.2526, -0.1415, -0.1962],
        [ 0.5191, -0.0852, -0.2043]], requires_grad=True)


In [29]:
sa_v1.wk.data.copy_(sa_v2.W_query.weight.T.clone())
sa_v1.wq.data.copy_(sa_v2.W_key.weight.T.clone())
sa_v1.wv.data.copy_(sa_v2.W_value.weight.T.clone())
# 因为sav1的权重是parameter方法创建的，包含了梯度等信息，如果想要更改的话，需要.data.copy_
# 并且由于可能存在内存冲突的情况，需要创建一个新的内存来复制

tensor([[ 0.2526,  0.5191],
        [-0.1415, -0.0852],
        [-0.1962, -0.2043]], grad_fn=<CopyBackwards>)

1️⃣ 为什么要用 .clone()？
原因1：防止“原地修改”影响源张量
很多 PyTorch 张量操作（如 .T、切片、view() 等）其实只是创建了原张量的“视图”，并没有真正拷贝数据。

直接用视图赋值/修改时，会影响原始数据，这可能会导致意外的副作用或梯度错误。

原因2：防止“内存重叠”导致的运行时错误
如果你对一个张量的某种变换（如转置、切片）赋值给另一个张量，如果二者的底层数据有内存重叠，PyTorch 会报错或警告。

clone() 会分配一块全新内存，消除重叠风险。

原因3：确保数据独立，可安全赋值或参与运算
对于copy_()、参数赋值、梯度操作等，强烈建议先 .clone()，保证数据之间没有引用、互不干扰。



In [30]:
sa_v1(inputs)

tensor([[-0.0809,  0.0638],
        [-0.0784,  0.0657],
        [-0.0784,  0.0657],
        [-0.0772,  0.0666],
        [-0.0787,  0.0652],
        [-0.0769,  0.0669]], grad_fn=<MmBackward0>)

In [31]:
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

## 3.5 使用因果注意力机制来屏蔽后续词

In [32]:
queries = sa_v2.W_query(inputs)                                   #A
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

#A 为了方便起见，我们复用上一节中 SelfAttention_v2 对象的query和key权重矩阵。

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [33]:
context_length = attn_weights.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [34]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [35]:
row_sum = mask_simple.sum(dim=-1,keepdim=True)
maskattn = mask_simple / row_sum
maskattn,row_sum

(tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]]),
 tensor([[1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.]]))

In [36]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [37]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [38]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
dropout(example)

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])

当对注意力权重矩阵应用 50% 的 dropout 时，矩阵中一半的元素会被随机设置为零。为了补偿有效元素的减少，矩阵中剩余元素的值会被放大 1/0.5 = 2 倍。这个缩放操作至关重要，可以在训练和推理阶段保持注意力机制的整体权重平衡，确保注意力机制在这两个阶段的平均影响保持一致。

In [39]:
torch.manual_seed(123)
attn_weights,dropout(attn_weights)

(tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
         [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
         [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
         [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
         [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
        grad_fn=<MulBackward0>))

In [40]:
batch = torch.stack((inputs,inputs),dim=0)
# 将两个inputs放入torch中模拟batch操作
# torch.cat 是“拼接”，只合并维度不增加新维度（比如把两个 [6, 3] 拼成 [12, 3] 或 [6, 6]）。
# torch.stack 是“叠加”，总是会新加一维。
# dim=0：在最前面加新维度（常用于 batch 处理）
# dim=1：在第1维插入（横向堆叠）
# dim=2：以此类推（更高维度）
batch,batch.size()

(tensor([[[0.4300, 0.1500, 0.8900],
          [0.5500, 0.8700, 0.6600],
          [0.5700, 0.8500, 0.6400],
          [0.2200, 0.5800, 0.3300],
          [0.7700, 0.2500, 0.1000],
          [0.0500, 0.8000, 0.5500]],
 
         [[0.4300, 0.1500, 0.8900],
          [0.5500, 0.8700, 0.6600],
          [0.5700, 0.8500, 0.6400],
          [0.2200, 0.5800, 0.3300],
          [0.7700, 0.2500, 0.1000],
          [0.0500, 0.8000, 0.5500]]]),
 torch.Size([2, 6, 3]))

In [41]:
# register_buffer 是 nn.Module（也就是你所有自定义模型和层的基类）里的一个方法。
# 它用来把“不是参数”（不需要梯度、不被优化器更新）但又要随着模型存储和迁移的张量“注册进模型”。

class CausalAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,bias=False):
        super().__init__()

        self.query = nn.Linear(d_in,d_out,bias=bias)
        self.key = nn.Linear(d_in,d_out,bias=bias)
        self.value = nn.Linear(d_in,d_out,bias=bias)
        self.dropout = nn.Dropout(dropout)
        self.output = d_out
        self.register_buffer(
            'mask',
            torch.triu(torch.ones((context_length,context_length)),diagonal=1)
        )


    def forward(self,input):
        batch_size,num_tokens,d_in = input.shape
        key = self.key(input)      # shape: [batch_size, num_tokens, d_out]
        query = self.query(input)  # shape: [batch_size, num_tokens, d_out]
        value = self.value(input)

        attn_socres = query @ key.transpose(1,2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens,:num_tokens],-torch.inf
        )
        attn_weight = torch.softmax(attn_scores / key.shape[-1]**0.5,dim = -1)
        attn_weight = self.dropout(attn_weight)
        context_vec = attn_weight @ value
        return context_vec
# 例如，当我们在大语言模型（LLM）中使用 CausalAttention 类时，buffer 会自动随模型迁移到合适的设备（CPU 或 GPU）。
# 这意味着我们无需手动确保这些张量与模型参数在同一设备上，从而避免设备不匹配错误

In [42]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])


##  3.6 从单头注意力扩展到多头注意力

In [43]:
# Listing 3.4 A wrapper class to implement multi-head attention
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

#nn.ModuleList是一个**专门用来存放子模块（layer/layer序列）**的Python列表，
# 继承自nn.Module。
# 它能让PyTorch自动追踪它里面的所有子模块（参数管理、模型保存/加载、迁移等都能正确工作）。

In [44]:
batch.size()

torch.Size([2, 6, 3])

In [45]:
context_length = batch.shape[1]
d_in=3
d_out=2
mha = MultiHeadAttentionWrapper(d_in,d_out,context_length,0.0,num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.4772, 0.1063, 0.4566, 0.2729],
         [0.5668, 0.2819, 0.5771, 0.3006],
         [0.6008, 0.3481, 0.6229, 0.3097],
         [0.5462, 0.3445, 0.5722, 0.2806],
         [0.5318, 0.3359, 0.5591, 0.2553],
         [0.5093, 0.3362, 0.5375, 0.2530]],

        [[0.4772, 0.1063, 0.4566, 0.2729],
         [0.5668, 0.2819, 0.5771, 0.3006],
         [0.6008, 0.3481, 0.6229, 0.3097],
         [0.5462, 0.3445, 0.5722, 0.2806],
         [0.5318, 0.3359, 0.5591, 0.2553],
         [0.5093, 0.3362, 0.5375, 0.2530]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [46]:
d_out = 1
mha = MultiHeadAttentionWrapper(d_in,d_out,context_length,0.0,num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.3749,  0.5063],
         [-0.1699,  0.6414],
         [-0.0860,  0.6925],
         [-0.0293,  0.6497],
         [ 0.0056,  0.5955],
         [ 0.0109,  0.5970]],

        [[-0.3749,  0.5063],
         [-0.1699,  0.6414],
         [-0.0860,  0.6925],
         [-0.0293,  0.6497],
         [ 0.0056,  0.5955],
         [ 0.0109,  0.5970]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [47]:
class MultiHeadAttnetion(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super().__init__()

        self.query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.keys = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.values = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length),diagonal=1)
        )
        assert d_out % num_heads == 0,"111"
        self.head_dim = d_out // num_heads
        self.dropout = nn.Dropout(dropout)
        self.d_out = d_out
        self.num_heads = num_heads
        self.out_proj = nn.Linear(d_out, d_out) 

    def forward(self,inputs):
        batch_size,num_tokens,d_in = inputs.shape

        queries = self.query(inputs).view(batch_size,num_tokens,self.num_heads,self.head_dim)
        keys= self.keys(inputs).view(batch_size,num_tokens,self.num_heads,self.head_dim)
        values = self.values(inputs).view(batch_size,num_tokens,self.num_heads,self.head_dim)

        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        # [batch_size,self.num_heads,num_tokens,self.head_dim]

        attn_scores = queries @ keys.transpose(2,3)
        ## [batch_size,self.num_heads,self.head_dim,num_tokens]
        # [batch_size,self.num_heads,num_tokens,num_tokens]
        # 维度变化 计算分数要求是[nums tokens * num tokens]
        mask_bool = self.mask.bool()[:num_tokens,:num_tokens]
        attn_scores.masked_fill_(mask_bool,-torch.inf)

        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vecs = (attn_weights @ values).transpose(1,2)
        # [batch_size,self.num_heads,num_tokens,self.head_dim]
        # 但是输出需要满足 batch_size,num_tokens,d_out 所以再次更换维度
        # [batch_size,num_tokens,self.num_heads，self.head_dim]
        context_vecs.contiguous().view(batch_size,num_tokens,d_out)
        context_vec = self.out_proj(context_vec)                  #K
        return context_vec


1. 单头注意力的局限性
单一注意力（只一组QKV权重）只能学会“某一种”关注规律，比如每个词对邻居的相关性。

但语言/图像/序列往往有多层面、多种远近关系需要被关注。

例：句子里既有短语依赖，又有长距离依赖。

2. 多头注意力怎么做的？
把总的输出维度d_out分成num_heads份，每份叫一个head，每个head有独立的Q/K/V权重（实际上你的代码实现有bug，下文会讲）。

每个head在自己的小空间里做注意力机制，可以捕捉不同类型、不同距离的信息模式。

最后把所有head的输出拼起来，用线性层混合成最终输出。

3. 带来的好处
多角度、多尺度感知：每个head有自己的注意力参数，可以关注不同内容、不同距离、不同特征。

信息融合：多个head的输出再混合，能“看得更全”，让模型学会同时捕捉多种语义/结构/空间信息。

更强泛化能力：单一注意力只能关注有限关系，多头能避免“注意力塌缩”问题。

并行高效：PyTorch设计得好，所有head同时并行计算，速度快。

In [48]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],             #A
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],
                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])
a.shape
#A 该张量的形状为 (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)

torch.Size([1, 2, 3, 4])

In [49]:
print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


In [50]:

first_head = a[0, 0, :, :]
print(first_head.shape)
first_res = first_head @ first_head.T
print("First head:\n", first_res)
second_head = a[0, 1, :, :]
print(second_head.shape)
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

torch.Size([3, 4])
First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])
torch.Size([3, 4])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])
